In [1]:
import os
import sys
import re
import time
import string
import unidecode
import requests
import datetime
import numpy as np
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [10]:
#loop to open the links
#then I gather the links inside each page

pages = list(range(0,460,30))
#loop

urls_list = []
for i in pages:
    complete_url = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=' + str(i)
    urls_list.append(complete_url)

In [11]:
#visualizing list
urls_list

['https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=0',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=30',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=60',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=90',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=120',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=150',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=180',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=210',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=240',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=270',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_start:int=300',
 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos?b_s

In [14]:
#scraping of the source code of the pages
#counter
counter = 16

#creating the function

def scraping_urls(urls):
    
    #define the driver
    driver = webdriver.Chrome()
    
    #list for the results
    soup_list = []
    
    #counter 
    count = 0
    
    #looping through the urls
    
    for i in urls:
        if count < counter:
            driver.get(i)
            driver.refresh()
            time.sleep(5)
            soup_list.append(BeautifulSoup(driver.page_source,'html.parser'))
        count += 1
    driver.close()
    return soup_list


In [15]:
#scraping the pages
source = scraping_urls(urls_list)

In [16]:
#cleaning the page's source and collecting all the links that lead us to the transcrition of the discourses

#just gathering the tags in the links

tag_links = []
for s in source:
    link=s.find_all('a', class_="summary url")
    tag_links.append(link)

In [17]:
#gathering the links inside the tags

links = []
for i in tag_links:
    for j in i:
        link=re.search(r'<a class="summary url" href=\"(.*?)" title="Document">', str(j)).group(1)
        links.append(link)

In [18]:
#scraping of the discourses' transcription

#defining a function to webscrap the text in the discourses in the links

def discourse_extractor(urls):
    driver = webdriver.Chrome()
    doc_source = []
    for i in  urls:
        driver.get(i)
        time.sleep(5)
        doc_source.append(BeautifulSoup(driver.page_source, 'html.parser'))
    driver.close()
    return doc_source


In [19]:
#appling the function to the list of links
discourses = discourse_extractor(links)

In [20]:
#extracting the date

def date_gather(soup_object):
    date_list=[]
    for s in soup_object:
        date=s.find_all('span',class_= 'documentPublished')
        date_list.append(date)
    return date_list

In [21]:
#loop to exctract the date

date_list=[]
for i in date_gather(discourses):
    date_list.append(len(i))

In [22]:
#exctracting the result of the gathering

dates = date_gather(discourses)

In [23]:
#loop to collect the date that appears on the span tag
clean_date=[]
for c in dates:
    cleaner= re.search(r'<span class="value"\>(.*?)</span>',str(c)).group(1)
    clean_date.append(cleaner)

#date without time
date_wt_time = []
for d in clean_date:
    wt_time=d.split(' ',1)[0]
    date_wt_time.append(wt_time)

In [24]:
#exctracting the text of the discourses from the links
#using beautifulsoup to find the html tags of each paragraph and save the
#texts in a list

def text_extractor(soup_object):
    text_list = []
    for s in soup_object:
        text = s.find_all('p')
        text_list.append(text)
    return text_list


In [25]:
#loop to extract the text
list_discourses = []
for i in text_extractor(discourses):
    list_discourses.append(len(i))


In [26]:
#exctracting discourses
first_discourses = text_extractor(discourses)

In [27]:
#verifying how many discourses were gathered
len(first_discourses)

475

In [28]:
#first discourses
first_discourses[0]

[<p><span><span> </span></span></p>,
 <p dir="ltr"><span>Senhores e senhoras,</span></p>,
 <p><span><span> </span></span></p>,
 <p dir="ltr" style="text-align: justify; "><span>É uma satisfação muito grande voltar a essa Casa, a qual eu integrei por 28 anos. Realmente rever amigos, conhecer novas pessoas, ver de perto novamente vocês, não tem preço.</span></p>,
 <p dir="ltr" style="text-align: justify; "><span>Nós só podemos pensar no Brasil melhor, a partir do momento que nós estivermos perfeitamente sintonizados. É dessa forma que se constrói uma grande nação e nós temos potencial para isso e muito mais. O mundo tem uma expectativa muito grande, o que nós temos aqui, o que nós produzimos e no que nós podemos fazer em parceria com todo mundo.</span></p>,
 <p dir="ltr" style="text-align: justify; "><span>Essa condecoração proposta pelo major Vitor Hugo, meu colega de Academia Militar das Agulhas Negras, muito me honra. Tô muito feliz nesse momento. Eu espero cada vez mais poder interag

In [30]:
#cleaning the text

#removing tags
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

strings=[]
for d in first_discourses:
    cleaned=remove_html_tags(str(d))
    strings.append(cleaned)

#removing \xa0 and defining everything lower case

clean = []
for i in strings:
    xa_remover=i.replace('\xa0', ' ').lower()
    clean.append(xa_remover)

In [31]:
#including space because there are cases
#where there is no space between the ponctuation and the words
#avoiding the concatenation of words

clean_space = []
for i in clean:
    esp=re.sub('([.,!?()])', r' \1 ', i)
    clean_space.append(esp)

In [33]:
#removing ponctuation
def ponctuation_remover(value):
    result = ""
    for c in value:
        if c not in string.punctuation:
            result += c
    return result

cleaner = []
for i in clean_space:
    remove_pon=ponctuation_remover(i)
    cleaner.append(remove_pon)

In [34]:
#removing withe space excess
#retirando excesso de espaços em branco
wt_space=[]
for i in cleaner:
    space_cleaner=i.replace("    ", " ").replace("   ", " ").replace("  ", " ")
    wt_space.append(space_cleaner)

In [35]:
#removing accents
final_text=[]
for i in wt_space:
    wt_accent=unidecode.unidecode(i)
    final_text.append(wt_accent)

In [36]:
#formatting the dates
final_dates=pd.to_datetime(date_wt_time, format='%d/%m/%Y')

In [37]:
#creating panda object with just the years
years=pd.DatetimeIndex(final_dates).year

In [39]:
#creating the dataframe

bolsonaro_discursos=pd.DataFrame([final_dates, years, links,final_text]).T
bolsonaro_discursos.columns=['data', 'ano', 'link', 'transcricao']
bolsonaro_discursos.insert(0,'presidente', 'bolsonaro')
bolsonaro_discursos.head(5)

,presidente,data,ano,link,transcricao
0,bolsonaro,2021-11-24,2021,https://www.gov.br/planalto/pt-br/acompanhe-o-...,senhores e senhoras e uma satisfacao muito gr...
1,bolsonaro,2021-11-16,2021,https://www.gov.br/planalto/pt-br/acompanhe-o-...,honrado e feliz estou no bahrein e parece que ...
2,bolsonaro,2021-11-11,2021,https://www.gov.br/planalto/pt-br/acompanhe-o-...,bom dia a todos menos para a primeiradama que...
3,bolsonaro,2021-11-10,2021,https://www.gov.br/planalto/pt-br/acompanhe-o-...,senhores e senhoras boa tarde a gente tem que...
4,bolsonaro,2021-11-07,2021,https://www.gov.br/planalto/pt-br/acompanhe-o-...,boa tarde a todos e uma satisfacao muito gran...


In [40]:
#creating a csv file
bolsonaro_discursos.to_csv('bolsonaro_discursos.csv')